In [6]:
import pandas as pd
import numpy as np

In [40]:
reserve_tb = pd.read_csv('./data/reserve.csv', encoding='UTF-8')

In [26]:
customer_tb = pd.read_csv('./data/customer.csv', encoding='UTF-8')

In [45]:
production_tb = pd.read_csv('./data/production.csv', encoding='UTF-8')

In [7]:
production_miss_num_tb = pd.read_csv('./data/production_missing_num.csv', encoding='UTF-8')

## 8. 수치형
### 8.1. 수치형 데이터로 변환

#### 다양한 수치형 데이터로 변환

In [4]:
type(40000 / 3)

float

In [5]:
int(40000 / 3)

13333

In [6]:
float(40000 / 3)

13333.333333333334

In [9]:
df = pd.DataFrame({'value': [40000 / 3]})
df.dtypes

value    float64
dtype: object

In [10]:
df['value']

0    13333.333333
Name: value, dtype: float64

In [11]:
df['value'].astype('int8')

0    21
Name: value, dtype: int8

In [12]:
df['value'].astype('int16')

0    13333
Name: value, dtype: int16

In [13]:
df['value'].astype('int32')

0    13333
Name: value, dtype: int32

In [14]:
df['value'].astype('int64')

0    13333
Name: value, dtype: int64

In [15]:
df['value'].astype('float16')

0    13336.0
Name: value, dtype: float16

In [16]:
df['value'].astype('float32')

0    13333.333008
Name: value, dtype: float32

In [17]:
df['value'].astype('float64')

0    13333.333333
Name: value, dtype: float64

In [18]:
df['value'].astype('float128')

0    13333.333333
Name: value, dtype: float128

In [19]:
df['value'].astype(int)

0    13333
Name: value, dtype: int64

In [20]:
df['value'].astype(float)

0    13333.333333
Name: value, dtype: float64

### 8.2. 대수화를 이용한 비선형 변화
대수화
- 입력값을 대수로 변환
- 값이 클수록 변화량을 작게 할 때 유용

> 나이를 대수화한 예  
> log(나이+1) = [대수화한 연령값]

#### 대수화

In [24]:
reserve_tb['total_price_log'] = \
    reserve_tb['total_price'].apply(lambda x: np.log10(x / 1000 + 1))
reserve_tb[['total_price', 'total_price_log']]

,total_price,total_price_log
0,97200,1.992111
1,20600,1.334454
2,33600,1.539076
3,194400,2.290925
4,68100,1.839478
...,...,...
4025,16000,1.230449
4026,41800,1.631444
4027,74800,1.879669
4028,540000,2.733197


### 8.3. 범주화를 이용한 비선형 변화
범주화
- 수치를 다수의 플래그값(true or false or etc)으로 변환하는 것
- 세세한 비연속 변화를 표현하려면 범주 수가 늘어나, 학습 데이터 양도 늘어남

#### 수치형의 범주화

In [34]:
customer_tb['age_rank'] = \
    (np.floor(customer_tb['age'] / 10) * 10).astype('category')
customer_tb[['age', 'age_rank']]
customer_tb['age_rank']

0      40.0
1      30.0
2      40.0
3      40.0
4      30.0
       ... 
995    40.0
996    30.0
997    30.0
998    40.0
999    30.0
Name: age_rank, Length: 1000, dtype: category
Categories (7, float64): [20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0]

### 8.4. 정규화
정규화
- 값이 가질 수 있는 범위를 통일하는 변환처리
- 입력값의 차이가 크면 생길 수 있는 과학습 등을 방지

방법
- 평균 0, 분산 1로 변환
 - ([입력값]-[입력값평균])/[입력값표준편차]
 - 튀는 값이 있을 경우 추천
- 최소값 0, 최대값 1로 변환 
 - ([입력값]-[입력값최소값])/([입력값최대값]-[입력값최소값])
 - 튀는 값이 없을 경우 추천

In [37]:
from sklearn.preprocessing import StandardScaler

reserve_tb['people_num'] = reserve_tb['people_num'].astype(float)

# 평균 0 분산 1 정규화
ss = StandardScaler()
# 최소 0 최대 1 정규화는 MinMaxScaler를 사용

result = ss.fit_transform(reserve_tb[['people_num', 'total_price']])

reserve_tb['people_num_normalized'] = [x[0] for x in result]
reserve_tb['total_price_normalized'] = [x[1] for x in result]

reserve_tb[[
    'people_num',
    'people_num_normalized',
    'total_price',
    'total_price_normalized'
]]

,people_num,people_num_normalized,total_price,total_price_normalized
0,4.0,1.300709,97200,-0.053194
1,2.0,-0.483753,20600,-0.747822
2,2.0,-0.483753,33600,-0.629935
3,4.0,1.300709,194400,0.828240
4,3.0,0.408478,68100,-0.317080
...,...,...,...,...
4025,2.0,-0.483753,16000,-0.789536
4026,2.0,-0.483753,41800,-0.555575
4027,2.0,-0.483753,74800,-0.256323
4028,4.0,1.300709,540000,3.962229


### 8.5. 예외값 제거
예외값
- 극단적으로 크거나 작은 값
- 특수한 상황을 분석해야 한다면 예외값을 제거하면 안 됨
- 정규본포 등의 계산식을 활용해 찾을 수 있음
- 식 없이 데이터를 가시화해서 직접 찾을 수도 있음

#### 표준편차 기준의 예외값 제거

In [43]:
reserve_tb_without_outlier = reserve_tb[(
    abs(
        reserve_tb['total_price'] -
        np.mean(reserve_tb['total_price'])
    ) / np.std(reserve_tb['total_price'])
    <= 3
)]
reserve_tb_without_outlier

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100
...,...,...,...,...,...,...,...,...,...
4024,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1,37200
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800


### 8.6. 주성분 분석을 이용한 차원 압축
주성분 분석
- 여러 종류의 입력값을 적은 종류의 입력값으로 압축
- 압축 후 차원 수는 기여율의 합계값(누적 기여율)을 기준으로 정함
- 데이터 가시화가 쉬움
- 새로운 차원에서 또다른 발견을 할 수도 있음
- 선형모델에서 과학습을 방지할 수 있음 (다른 방법도 많음)
- 비선형모델에서 정보 손실로 정확도가 떨어질 수 있음

#### 주성분 분석을 이용한 차원 압축

In [49]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_values = pca.fit_transform(production_tb[['length', 'thickness']])

print('누적기여율 {0}'.format(sum(pca.explained_variance_ratio_)))
print('각차원기여율 {0}'.format(pca.explained_variance_ratio_))

pca_newvalues = pca.transform(production_tb[['length', 'thickness']])
pca_newvalues

누적기여율 1.0
각차원기여율 [0.97897794 0.02102206]


array([[  76.96838157,  -13.38906936],
       [-112.11469337,   -8.24884796],
       [ -76.1994339 ,   11.19027127],
       ...,
       [  31.12100559,   15.48152593],
       [-117.87675543,    2.4361334 ],
       [   4.80243541,  -15.32174872]])

### 8.7. 수치의 보완

- 정수로 보완  
결손값이 많을 때는 비추천
- 집계값으로 보완  
평균값/중앙값/최소값/최대값
결손값이 많을 때는 비추천
- 결손되지 않은 데이터를 이용한 보완
- 시간관계를 이용해 보완
- 다중대입법으로 보완  
보완한 데이터셋을 여러 개 만들어 각 데이터셋을 해석
- 최대 가능도로 보완  
잠재 변수를 도입해 EM 알고리즘을 사용, 가능도를 최대화하여 결손값을 보완

#### 결손 레코드 제거

In [58]:
production_miss_num = production_miss_num_tb.copy()
production_miss_num.replace('None', np.nan, inplace=True)
production_miss_num.dropna(subset=['thickness'], inplace=True)

#### 정수 보완

In [59]:
production_miss_num = production_miss_num_tb.copy()
production_miss_num.replace('None', np.nan, inplace=True)
production_miss_num['thickness'].fillna(1, inplace=True)

#### 평균값 보완

In [60]:
production_miss_num = production_miss_num_tb.copy()
production_miss_num.replace('None', np.nan, inplace=True)
production_miss_num['thickness'] = \
    production_miss_num['thickness'].astype('float64')
thickness_mean = production_miss_num['thickness'].mean()
production_miss_num['thickness'].fillna(thickness_mean, inplace=True)

#### PMM을 이용한 다중대입

In [9]:
from fancyimpute import IterativeImputer as MICE

production_miss_num = production_miss_num_tb.copy()
production_miss_num.replace('None', np.nan, inplace=True)

production_miss_num['thickness'] = \
    production_miss_num['thickness'].astype('float64')
production_miss_num['type'] = \
    production_miss_num['type'].astype('category')
production_miss_num['fault_flg'] = \
    production_miss_num['fault_flg'].astype('category')
    
production_dummy_flg = pd.get_dummies(
    production_miss_num[['type', 'fault_flg']], drop_first=True
)

# FIXME
mice = MICE(n_imputations=10, n_burn_in=50, impute_type='pmm')

production_mice = mice.multiple_imputations(
    pd.concat(
        [
            production_miss_num[['length', 'thickness']],
            production_dummy_flg
        ],
        axis=1
    )
)

production_mice[0]

TypeError: __init__() got an unexpected keyword argument 'n_burn_in'